In [6]:
pip install psycopg2-binary


Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import psycopg2

# Your DB config
db_config = {
    "host": "172.16.20.153",
    "port": 5432,
    "database": "gis_db",
    "user": "qurbanzada.emil",
    "password": "P#mnsgWdH5654!"
}

# File path to the GeoJSON file
geojson_file_path = r"/Users/username/Desktop/GeoJSON_2/Attached_geojson/Attached_properties.geojson"

# Load GeoJSON
with open(geojson_file_path, "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Connect to PostgreSQL
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Prepare insert statement
insert_sql = """
    INSERT INTO public.valuation_layout (
        special_co,
        title_by_document,
        total_area,
        owner,
        category,
        subcategory,
        type,
        geometry_coordinates
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

def convert_flat_list_to_multipolygon(flat_list):
    """
    Converts flat list [lon1, lat1, lon2, lat2, ...] into [[[ [lon, lat], ... ]]]
    """
    if not flat_list or len(flat_list) % 2 != 0:
        return None

    ring = []
    for i in range(0, len(flat_list), 2):
        ring.append([flat_list[i], flat_list[i + 1]])

    # Ensure ring is closed
    if ring[0] != ring[-1]:
        ring.append(ring[0])

    return ring


# Insert each feature
for feature in geojson_data["features"]:
    props = feature["properties"]
    geom_type = feature["geometry"]["type"]
    coords = feature["geometry"]["coordinates"]

    # Fix: Convert flat coordinate list (lon, lat, ...) to MultiPolygon
    if isinstance(coords[0], (int, float)):
        geometry_coordinates = convert_flat_list_to_multipolygon(coords)
    elif geom_type == "Polygon":
        geometry_coordinates = [[coords[0]]]  # wrap polygon as multipolygon
    elif geom_type == "MultiPolygon":
        geometry_coordinates = coords
    else:
        geometry_coordinates = None

    values = (
        str(props.get("id")),
        props.get("Name"),
        float(props.get("Sahə ( ha )", 0)) * 10000,
        props.get("Hüquqi sahibin adı"),
        props.get("category"),
        props.get("subcategory"),
        "MultiPolygon",
        geometry_coordinates
    )

    cursor.execute(insert_sql, values)


# Commit and close
conn.commit()
cursor.close()
conn.close()
